In [1]:
import pandas as pd
import numpy as np
import import_ipynb
from calc_features import calc_all_features
from calc_features import clean_data
from calc_features import ls_need_col as ls

importing Jupyter notebook from calc_features.ipynb


In [8]:
ls_need_col = ls.copy()
# df_prepared_merge = clean_data('prepared_hard.xlsx', ls_need_col)
# df_prepared_valid = clean_data('prepared_hard_valid.xlsx', ls_need_col)
# ls_need_col.remove('t˚ C трубы после ванны')
# ls_need_col.remove('время выдержки в закалочной ванне, сек.')
df_prepared_merge = pd.read_excel('prepared_hard.xlsx')
df_prepared_valid = pd.read_excel('prepared_hard_valid.xlsx')

df_prepared_merge = df_prepared_merge.reset_index(drop=True)
df_prepared_valid = df_prepared_valid.reset_index(drop=True)

In [9]:
df_valid = df_prepared_valid.copy()
df_merge = df_prepared_merge.copy()

In [4]:
df_prepared_valid.columns

Index(['1 зона по ВТР закалка', '2 зона по ВТР закалка',
       '3 зона по ВТР закалка',
       'Скорость прохождения трубы через спрейер, м/с',
       't˚ C трубы после спреера', '1 зона ВТР и уставка отпуск',
       '2 зона ВТР и уставка отпуск', '3 зона ВТР и уставка отпуск',
       '4 зона ВТР и уставка отпуск', '5 зона ВТР и уставка отпуск',
       'шаг балок закалочная печь, сек', 'шаг балок отпускная печь, сек',
       'диаметр', 'толщина стенки', 'C', 'Mn', 'Si', 'P', 'S', 'Cr', 'Ni',
       'Cu', 'Al', 'V', 'Ti', 'Nb', 'Mo', 'N', 'Параметр закалка',
       'Параметр отпуск новый', 'C-coef', 'ICD', 'Предел текучести',
       'Врем. сопротивление', '№ партии'],
      dtype='object')

In [5]:
ls_modes = [
 '1 зона по ВТР закалка',
 '2 зона по ВТР закалка',
 '3 зона по ВТР закалка',
 'Скорость прохождения трубы через спрейер, м/с',
 't˚ C трубы после спреера',
 '1 зона ВТР и уставка отпуск',
 '2 зона ВТР и уставка отпуск',
 '3 зона ВТР и уставка отпуск',
 '4 зона ВТР и уставка отпуск',
 '5 зона ВТР и уставка отпуск',
 'шаг балок закалочная печь, сек',
 'шаг балок отпускная печь, сек',
 'диаметр',
 'толщина стенки'
]

In [6]:
df_prepared_merge.reset_index(drop=True, inplace=True)
df_prepared_valid.reset_index(drop=True, inplace=True)

In [7]:
df_prepared_valid=df_prepared_valid[ls_need_col+['Предел текучести', 'Врем. сопротивление', '№ партии']]
df_prepared_merge=df_prepared_merge[ls_need_col+['Предел текучести', 'Врем. сопротивление', '№ партии']]

KeyError: "['время выдержки в закалочной ванне, сек.' 't˚ C трубы после ванны'] not in index"

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
df = pd.concat([df_prepared_valid, df_prepared_merge])
df.reset_index(drop=True, inplace=True)

In [40]:
kmeans = KMeans(n_clusters=6000, init='k-means++', verbose=0).fit(df[ls_modes])

In [41]:
df['kmeans_label'] = kmeans.labels_

In [43]:
def mean_if_norm(x):
    for col in ['Предел текучести', 'Врем. сопротивление']:
        if max(x[col]) - min(x[col]) >= 9.0:
            x[col] = None
        else:
            x[col] = x[col].mean()
    return x.mean()
df_try = df.groupby('kmeans_label')[ls_need_col+['Предел текучести', 'Врем. сопротивление']].apply(np.mean).dropna()

In [44]:
df_try.to_excel('try_kmeans.xlsx')

In [6]:
# df_prepared_valid.index =pd.MultiIndex.from_arrays([
#     df_prepared_valid['1 зона по ВТР закалка'],
#     df_prepared_valid['2 зона по ВТР закалка'],
#     df_prepared_valid['3 зона по ВТР закалка'],
#     df_prepared_valid['Скорость прохождения трубы через спрейер, м/с'],
#     df_prepared_valid['t˚ C трубы после спреера'],
#     df_prepared_valid['1 зона ВТР и уставка отпуск'],
#     df_prepared_valid['2 зона ВТР и уставка отпуск'],
#     df_prepared_valid['3 зона ВТР и уставка отпуск'],
#     df_prepared_valid['4 зона ВТР и уставка отпуск'],
#     df_prepared_valid['5 зона ВТР и уставка отпуск'],
#     df_prepared_valid['шаг балок закалочная печь, сек'],
#     df_prepared_valid[ 'шаг балок отпускная печь, сек'],
#     df_prepared_valid['диаметр'],
#     df_prepared_valid['толщина стенки']
# ])

In [19]:
# df_prepared_merge.groupby(ls_modes)[['Предел текучести', 'Врем. сопротивление', 'C-coef']].describe().to_excel('describe_train_modes_easy.xlsx')